# **Oficina 6: Desenvolvendo um sistema de classificação**

- Base de Dados: [Spam Emails - Kaggle](https://www.kaggle.com/datasets/abdallahwagih/spam-emails)

In [1]:
# Importações
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import joblib

## Base de dados

In [2]:
# Carregando o dataset
df = pd.read_csv('./data/spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# informações sobre o dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
# verificando a distribuição dos dados entre spam e ham
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

## Limpeza e Preparação de Dados

In [5]:
# Baixando a lista de stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Criando um função de pré-processamento
def preprocessamento(texto):
    # Deixar o texto em caixa baixa
    texto = texto.lower()
    # Remover links
    texto = re.sub(r"http\S+|www\S+|https\S+", '', texto, flags=re.MULTILINE)
    # Remover tags HTML
    texto = re.sub(r'<.*?>', '', texto)
    # Remover pontuações
    texto = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', texto)
    # Remover stopwords
    texto = ' '.join([palavra for palavra in texto.split() if palavra not in stopwords.words('english')])
    return texto

<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\]'
C:\Users\joaoi\AppData\Local\Temp\ipykernel_8236\2148229579.py:10: SyntaxWarning: invalid escape sequence '\]'
  texto = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', texto)


In [7]:
# Aplicando a função de pré-processamento
df['Message'] = df['Message'].apply(preprocessamento)

In [8]:
# Transformando os textos em vetores
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Message']).toarray()

# Definindo as variáveis dependentes e independentes
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})
y = df['Category'].values
y

array([0, 0, 1, ..., 0, 0, 0], shape=(5572,))

## Divisão em Treino e Teste

In [9]:
# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4457, 8531), (1115, 8531), (4457,), (1115,))

In [10]:
# Treinando o modelo
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [11]:
# Realizando as previsões
y_pred = model.predict(X_test)

## Avaliando o Modelo

In [12]:
# Visualizando a matriz de confusão
print(confusion_matrix(y_test, y_pred))

[[966   0]
 [ 29 120]]


In [13]:
# Visualizando o relatório de classificação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       966
           1       1.00      0.81      0.89       149

    accuracy                           0.97      1115
   macro avg       0.99      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



- O modelo está muito bom (97% de acurácia).
- No entanto, o recall da classe "spam" (0.81) sugere que ele pode estar deixando passar alguns e-mails de spam.